In [5]:
import numpy as np
import pandas as pd
from pymongo import MongoClient
from datetime import datetime, timedelta

In [4]:
def pulling_data_db():
    client = MongoClient('localhost', 27017)
    db = client['financeStockData']
    collection = db['stockData']
    
    dt = collection.find()
 
    fin_data = pd.DataFrame(dt)
    return fin_data

In [5]:
df = pulling_data_db()
df

""


In [10]:
from pymongo import MongoClient
import pandas as pd

# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client['financeStockData']
collection = db['stock_data']

# Retrieve data from MongoDB collection
data = collection.find()

# Convert MongoDB cursor to a list of dictionaries
data_list = list(data)

df = pd.DataFrame(data_list)
df.head()


,_id,Open,High,Low,Close,Adj Close,Volume
0,670fce3a0f9694709cdfea07,187.149994,188.440002,183.889999,185.639999,184.938217,82488700
1,670fce3a0f9694709cdfea08,184.220001,185.880005,183.429993,184.250000,183.553467,58414500
2,670fce3a0f9694709cdfea09,182.149994,183.089996,180.880005,181.910004,181.222336,71983600
3,670fce3a0f9694709cdfea0a,181.990005,182.759995,180.169998,181.179993,180.495071,62303300
4,670fce3a0f9694709cdfea0b,182.089996,185.600006,181.500000,185.559998,184.858521,59144500
